# ACHTUNG!!
translate(Hausaufgabe) -> **Homework**


на тему Seq2Seq модели в машинном переводе и

Halo!
На семинаре мы создали простую seq2seq модель на основе rnn для перевода, а сейчас постараемся засунуть туда attention. Работать будем с тем же датасетом DE->EN (датасеты получше просто не влезают в память колаба, но если у вас есть CPU+тонна времени или GPU побольше, то можно попробовать построить перевод на WMT14 или IWSLT )


In [0]:

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import spacy

import random
import math
import time

from torchtext.datasets import TranslationDataset, Multi30k #WMT14, IWSLT
from torchtext.data import Field, BucketIterator

import torch.nn.functional as F

In [0]:
seed = 43

random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
! python -m spacy download en
! python -m spacy download de


spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [0]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

# немецкий язык является полем SRC, а английский в поле TRG
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [0]:
# В датасете содержится ~ 30к предложений средняя длина которых 11
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'),  fields = (SRC, TRG))

Давайте посмотрим что у нас с датасетом и сделаем словари для SRC и TGT

In [0]:
labels = ['train', 'validation', 'test']
dataloaders = [train_data, valid_data, test_data]
for d, l in zip(dataloaders, labels):
    print("Number of sentences in {} : {}".format(l, len(d.examples)))


In [0]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)
print("Number of words in source vocabulary", len(SRC.vocab))
print("Number of words in source vocabulary", len(TRG.vocab))

## Encoder

Энкодер будет ровно как в семинаре, с кдинственным изменением -- forward будет возвращать не только hidden, cell, но еще и outputs. Это нужно (надеюсь, вы уже поняли) для использования attention в декодере

In [0]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        """
        :param: input_dim is the size/dimensionality of the one-hot vectors that will be input to the encoder. This is equal to the input (source) vocabulary size.
        :param: emb_dim is the dimensionality of the embedding layer. This layer converts the one-hot vectors into dense vectors with emb_dim dimensions.
        :param: hid_dim is the dimensionality of the hidden and cell states.
        :param: n_layers is the number of layers in the RNN.
        :param: percentage of the dropout to use
        
        """
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)
 
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        """
        :param: src sentences (src_len x batch_size)
        """
        # embedded = <TODO> (src_len x batch_size x embd_dim)
        embedded = self.embedding(src)
        # dropout over embedding
        embedded = self.dropout(embedded)
        outputs, (hidden, cell) = self.rnn(embedded)
        # [Attention return is for lstm, but you can also use gru]
        return outputs, hidden, cell

## Decoder
Оп ля, а тут уже что-то новенькое

Мы будем реализовывать attention, который будет смотреть из tgt в src (НЕ self-attention). 

Определим два класса -- Attention и DecoderAttn. Мы разбили их на два класса, чтобы можно было играться с типом внимания, не меняя код DecoderAttn. Как вы помните с лекции, в качестве аттеншена можно брать любую странную функцию (конкатенация, маленькая сеточка, ...), и все будет работать! Поэтому вам предлагается попробовать несколько разных.


---------------------
Есть два подхода к реализации аттеншена:

Подход #1:

1. Вычисляется embed
2. На основе hidden c прошлого шага, embedded и (возможно) enc_out вычисляется attention, а точнее, веса attention (поэтому не забудьте softmax!!). Размерность batch_size * max_len, max_len -- максимальная длина предложения в батче, т.е. shape[0] от выхода энкодера.
3. К enc_out применяется attention: чаще всего dot product от enc_out и attention_weights (не забудьте про измерение батч. Чтобы нормально вычислить dot_product по батчу, вам поможет torch.bmm)
4. Берутся attention и embedded и сворачиваются в один вектор размерности такой, чтобы кормить его self.lstm. Например, это можно сделать с помощью обычного линейного слоя
5. Вычисляется новое скрытое состояние new_hidden. Это наша self.lstm, примененная к выходу пункта 4.
6. Вычисляется prediction, как в семинаре

Грубо говоря, вся разница с семинаром в том, что мы вместо того, чтобы embedded пихать в self.lstm, миксуем аттэншен на основе всего, что имеем (enc_out, hidden, embedded) и запихиваем в self.lstm микс аттэншена и embedded.

![alt text](https://i.imgur.com/cmkRY0r.png)


Подход #2:

1. Вычисляется embed
2. Вычисляется output, new_hidden (строчка output, (hidden, cell) = self.rnn(embedded, (hidden, cell)))
3. На основе output и enc_out вычисляется attention, а точнее, веса attention (поэтому не забудьте softmax!!)
3. К enc_out применяется attention: чаще всего dot product от enc_out и attention_weights (не забудьте про измерение батч. Чтобы нормально вычислить dot_product по батчу, вам поможет torch.bmm)
4. Вычисляется prediction на основе attention и output. Можно, например, взять nn.Linear() от конкатенации attention и output.

Разница с первым подходом в том, что мы сначала вычисляем выход rnn слоя, а потом смотрим вниманием на src и на основе выхода rnn и attn считаем выход (prediction). 

![alt text](https://i.imgur.com/5aWjQWv.png)


Вам предлагается реализовать хотя бы 1 из вариантов и хотя бы 2 варианта функции attention (в классе Attention)


In [0]:
class Attention(nn.Module):
    def __init__(self, batch_size, method="one-layer-net"): # add parameters needed for your type of attention
        super(Attention, self).__init__()
        self.method = method # attention method you'll use. e.g. "cat", "one-layer-net", "dot", ...
        
        <YOUR CODE HERE>
        
        def forward(self, last_hidden, encoder_outputs, seq_len=None):
            
            <YOUR CODE HERE>
            raise NotImplementedError
        
        
      
class DecoderAttn(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, attention, dropout=0.1):
        super(DecoderAttn, self).__init__()
        
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        
        self.attn = attention # instance of Attention class
        
        self.dropout = dropout

        # define layers
        self.embedding = nn.Embedding(self.output_dim, self.emb_dim)
        
        self.rnn = nn.LSTM(self.emb_dim, self.hid_dim, self.n_layers) #(lstm embd, hid, layers, dropout)
        self.out = nn.Linear(self.hid_dim, self.output_dim) # Projection :hid_dim x output_dim
        self.dropout = nn.Dropout(dropout)

        # more layers you'll need for attention
        <YOUR CODE HERE>
        
    def forward(self, input, hidden, cell, encoder_output):
        # make decoder with attention
        # use code from seminar notebook as base and add attention to it
        <YOUR CODE HERE>
        
        
        return prediction, hidden, cell

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Seq2Seq module

Здесь опять ничего не поменяется кроме того, что энкодер теперь возвращает свой output, а декодер его принимает

In [0]:
# we need that to put it first to the decoder in 'translate' method
BOS_IDX = SRC.vocab.stoi['<sos>']

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        # Hidden dimensions of encoder and decoder must be equal
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self._init_weights() 
        self.max_len=30
    
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        """
        :param: src (src_len x batch_size)
        :param: tgt
        :param: teacher_forcing_ration : if 0.5 then every second token is the ground truth input
        """
        
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        enc_out, hidden, cell = self.encoder(src, ) # TODO pass src throw encoder
        
        #first input to the decoder is the <sos> tokens
        input = trg[0, :] # TODO trg[idxs]
        
        for t in range(1, max_len):
            
            output, hidden, cell = self.decoder(input, hidden, cell, enc_out) #TODO pass state and input throw decoder 
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            input = (trg[t] if teacher_force else top1)
        
        return outputs
    
    def translate(self, src):
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = []
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        src = torch.tensor(src).to(self.device)
        hidden, cell = self.encoder(src.reshape((src.shape[0], 1))) # TODO pass src throw encoder
        
        #first input to the decoder is the <sos> tokens
        input = torch.tensor([BOS_IDX]).to(self.device)# TODO trg[idxs]
        
        for t in range(1, self.max_len):
            
            output, hidden, cell = self.decoder(input, hidden, cell) #TODO pass state and input throw decoder 
            top1 = output.max(1)[1]
            outputs.append(top1)
            input = (top1)
        
        return outputs
    
    def _init_weights(self):
        p = 0.08
        for name, param in self.named_parameters():
            nn.init.uniform_(param.data, -p, p)
        


In [0]:
input_dim = len(SRC.vocab)
output_dim = len(TRG.vocab)
src_embd_dim =  tgt_embd_dim = 256
hidden_dim = 512
num_layers =  2
dropout_prob = 0.2

batch_size = 64
PAD_IDX = TRG.vocab.stoi['<pad>']

iterators = BucketIterator.splits((train_data, valid_data, test_data),
                                  batch_size = batch_size, device = device)
train_iterator, valid_iterator, test_iterator = iterators

enc = Encoder(input_dim, src_embd_dim, hidden_dim, num_layers, dropout_prob)
dec = DecoderAttn(output_dim, tgt_embd_dim, hidden_dim, num_layers, dropout_prob)
model = Seq2Seq(enc, dec, device).to(device)


In [0]:
model

In [0]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        print(batch.src.shape)
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing !!
            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)


            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [0]:
max_epochs = 100
CLIP = 1

# TODO
optimizer = optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

best_valid_loss = float('inf')

for epoch in range(max_epochs):
    
    
    train_loss = round(train(model, train_iterator, optimizer, criterion, CLIP), 5)
    valid_loss = round(evaluate(model, valid_iterator, criterion),5)
    
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print('Epoch: {} \n Train Loss {}  Val loss {}:'.format(epoch, train_loss, valid_loss))
    print('Train Perplexity {}  Val Perplexity {}:'.format(np.exp(train_loss), np.exp(valid_loss)))


In [0]:
test_loss = evaluate(model, test_iterator, criterion)

print('| Test Loss: {} Test PPL:{}|'.format(test_loss, np.exp(test_loss)))

In [0]:
EOS_IDX = SRC.vocab.stoi['<eos>']

def translate(sentence):
    """
    function that uses .translate() method of the model to translate german sentence into english
    params:
        sentence: tokenized gernam sentence
    """
    sentence = sentence.lower()
    sent_vec = [SRC.vocab.stoi[token] for token in sentence.split()]
    translation_idx = model.translate(torch.tensor(sent_vec))
    for t in translation_idx:
        if t[0] != EOS_IDX:
            print(TRG.vocab.itos[t[0]], end=' ')
        else:
            break

In [0]:
translate("ein klein apfel")

ИИИИ давайте также научимся считать самую популярную метрику для перевода -- BLEU (https://en.wikipedia.org/wiki/BLEU)

В общем-то, вам повезло -- ее писать руками скучно, да и nltk ее написало за вас:

In [0]:
from nltk.translate.bleu_score import corpus_bleu
def compute_bleu(inp_lines, out_lines):
    """ Estimates corpora-level BLEU score of model's translations given inp and reference out """
    translations = [translate(line) for line in inp_lines]
    return corpus_bleu([[ref] for ref in out_lines], translations) * 100

In [0]:
compute_bleu(<batch_of_sentences>, <batch_of_sentences>)